In [1]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple

In [2]:
with open("queries.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]

In [3]:
queries2results = dict()

In [4]:
dataset_name = "covid19"
num_results = 100
query_type = "syntactic"

for q in tqdm.tqdm(queries, total = len(queries)):
    df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
    queries2results[q] = df

100%|██████████| 67/67 [06:09<00:00,  5.51s/it]


In [5]:
with open("queries2results.pickle", "wb") as f:
    pickle.dump(queries2results, f)

In [4]:
with open("queries2results.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [5]:
data = []
for q in queries:
    #print(q)
    results = queries2results[q]
    sents, arg1_first, arg2_first = results["sentence_text"].tolist(), results["arg1_first_index"].tolist(), results["arg2_first_index"].tolist()
    arg1_last, arg2_last = results["arg1_last_index"].tolist(), results["arg2_last_index"].tolist()
    
    sents_with_args = []
    for s,arg1_ind,arg2_ind, arg1_ind_last, arg2_ind_last in zip(sents,arg1_first,arg2_first, arg1_last, arg2_last):
        s_lst = s.split(" ")
        if arg1_ind > arg2_ind:
            arg1_ind, arg2_ind = arg2_ind, arg1_ind
            arg1_ind_last, arg2_ind_last = arg2_ind_last, arg1_ind_last
            arg1_str, arg2_str = "ARG2:", "ARG1:"
        else:
            arg1_str, arg2_str = "ARG1:", "ARG2:"
        s_with_args = s_lst[:arg1_ind] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args)
        sents_with_args.append({"sent": s_with_args, "start_1": arg1_ind, "start_2": arg2_ind, "end_1": arg1_ind_last,
                               "end_2": arg2_ind_last})
    
    max_number_of_pairs = 100
    pairs = list(itertools.product(sents_with_args, repeat=2))
    print(len(pairs))
    random.shuffle(pairs)
    for pair in pairs[:max_number_of_pairs]:
        data.append({"first": pair[0]["sent"], "second": pair[1]["sent"], "query": q, "first_arg1": (pair[0]["start_1"], pair[0]["end_1"]),
                    "first_arg2": (pair[0]["start_2"], pair[0]["end_2"]), "second_arg1": (pair[1]["start_1"], pair[1]["end_1"]),
                   "second_arg2": (pair[1]["start_2"], pair[1]["end_2"])})

10000
1225
10000
10000
36
10000
10000
10000
81
10000
10000
10000
10000
10000
2601
10000
5041
10000
10000
576
10000
4096
10000
10000
169
6241
10000
4
10000
841
3721
10000
10000
441
2704
169
10000
3969
361
10000
10000
10000
576
10000
16
25
196
10000
10000
576
169
3600
529
3844
10000
10000
10000
576
576
10000
121
25
529
289
10000
2916
10000


In [6]:
random.shuffle(data)
data[0]

with open("data.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [9]:
import json

with open("data.txt", "w") as f:
    for d in data:
        first, second = d["first"], d["second"]
        first_arg1 = d["first_arg1"]
        first_arg2 = d["first_arg2"]
        second_arg1 = d["second_arg1"]
        second_arg2 = d["second_arg2"]
        
        elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
        keys = ["first", "second", "first_arg1", "first_arg2"]
        
        f.write(json.dumps(d) + "\n")
        #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")

In [10]:
random.shuffle(data)
data[9]

{'first': '3e6 According to the report of the WHO and China Joint Mission on Coronavirus Disease 2019 ( COVID- 19 ) , it is a zoonotic virus , based on the data available , ARG1:bats seems to be the reservoir of COVID-19 ARG2:virus .',
 'second': 'Furthermore , sequence analyses indicated the existence of a much greater genetic diversity of SARS-like-CoVs in bats than of SARS-CoVs in civets or humans , which supports the notion that SARS-CoV is a member of this novel coronavirus group and that ARG1:bats are a natural reservoir for ARG2:it .',
 'query': 'arg1:[e]bats are a $reservoir of arg2:[e]something',
 'first_arg1': (33, 33),
 'first_arg2': (41, 41),
 'second_arg1': (41, 41),
 'second_arg2': (47, 47)}

In [11]:
queries2results[queries[0]]

,sentence_id,arg1,arg2,title,article_link,pmc_id,doi,pdf_sha,pubmed_id,msft_id,arg1_first_index,arg1_last_index,arg2_first_index,arg2_last_index,sentence_text,paragraph_text
0,8903,Mouse,depletion,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,0,8,8,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
1,8903,Mouse,macrophages,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,0,13,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
2,8903,Mouse,alterations,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,0,17,17,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
3,8903,hepatitis,depletion,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,1,1,8,8,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
4,8903,hepatitis,macrophages,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,1,1,13,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,540072,Rhinovirus,cells,Rhinoviruses,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7150364,10.1016/b978-012373960-5.00610-9,NaN,NaN,NaN,0,0,7,7,Rhinovirus infection causes lysis of the infec...,Rhinovirus has a propensity to affect young ch...
96,540072,Rhinovirus,decrease,Rhinoviruses,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7150364,10.1016/b978-012373960-5.00610-9,NaN,NaN,NaN,0,0,10,10,Rhinovirus infection causes lysis of the infec...,Rhinovirus has a propensity to affect young ch...
97,546506,COVID-19,cytokine,Comparison of confirmed COVID‐19 with SARS and...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7300470,10.1002/rmv.2112,NaN,32502331.0,NaN,6,6,18,18,128 Another study reported that following COVI...,Most of the patients with coronavirus had abno...
98,546506,COVID-19,storm,Comparison of confirmed COVID‐19 with SARS and...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7300470,10.1002/rmv.2112,NaN,32502331.0,NaN,6,6,19,19,128 Another study reported that following COVI...,Most of the patients with coronavirus had abno...


In [75]:
class BertModel(pl.LightningModule):

    def __init__(self, train_dataset: Dataset, dev_dataset: Dataset, batch_size, device: str, mode: str = "eval"):
        
        super().__init__()
        
        self.device = device
        config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        self.model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config)    
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        
        if mode == "eval":
            
            self.model.eval()
        else:
            self.model.train()
            
        self.train_gen = torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_dataset, batch_size=batch_size, drop_last=False, shuffle=False)
        self.acc = None

        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        tok_to_orig_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        tok_to_orig_map = {}

        keys = list(sorted(orig_to_tok_map.keys()))
        for k, k2 in zip(keys, keys[1:]):
            for k3 in range(orig_to_tok_map[k], orig_to_tok_map[k2]):
                tok_to_orig_map[k3] = k

        bert_tokens.append("[SEP]")
        
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(bert_tokens)
        tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)

        return (bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor)
    
    
    def forward(self, x):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        return states
    
    def forward_with_loss_calculation(self, x, sent1_idx, sent2_idx, orig_to_tok_map, l):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        states = states/torch.norm(states, dim = 1, keepdim = True)
        
        gold_arg1_sent1 = orig_to_tok_map[sent1_idx[0].detach().cpu().numpy().item()]
        gold_arg2_sent1 = orig_to_tok_map[sent1_idx[1].detach().cpu().numpy().item()]
        
        sent1_arg1_vec, sent1_arg2_vec = states[gold_arg1_sent1], states[orig_to_tok_map[gold_arg2_sent1]
        
        #states_sent2 = states[orig_to_tok_map[l]:]  
        
        sims_arg1 = sent1_arg1_vec @ states.T
        sims_arg2 = sent1_arg2_vec @ states.T
        
        gold_arg1_sent2, gold_arg2_sent2 = orig_to_tok_map[sent2_idx[0].detach().cpu().numpy().item()], orig_to_tok_map[sent2_idx[1].detach().cpu().numpy().item()]
        sim_arg1_gold = sims_arg1[gold_arg1_sent2]
        sim_arg2_gold = sims_arg2[gold_arg2_sent2]
        
        idx_arg1 = torch.argsort(sims_arg1)
        idx_arg2 = torch.argsort(sims_arg2)
        
        most_sim_arg1, most_sim_arg2 = sims_arg1[idx_arg1[-2]], sims_arg2[idx_arg2[-2]]
        
        alpha = 0.1
        loss_arg1 = torch.max(torch.zeros(1).to(self.device), -sim_arg1_gold + most_sim_arg1 + alpha)
        loss_arg2 = torch.max(torch.zeros(1).to(self.device), -sim_arg2_gold + most_sim_arg2 + alpha)
                                                                         
        if idx_arg1[-2] == gold_arg1_sent2:
            loss_arg1 = torch.zeros(1).to(self.device)
                                                                        
        if idx_arg2[-2] == gold_arg2_sent2:
            loss_arg2 = torch.zeros(1).to(self.device)                                                      
                                                                         
        loss = loss_arg1 + loss_arg2
        
        return loss
    
    def training_step(self, batch, batch_nb):
        
        sents_concat, sent1_idx, sent2_idx, l = batch  
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))        
        loss = self.forward_with_loss_calculation(tokens_tensor, sent1_idx, sent2_idx, orig_to_tok_map, l)

        return {'loss': loss}
    
    def validation_step(self, batch, batch_nb):

        sents_concat, sent1_idx, sent2_idx, l = batch
        
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat[0].split(" "))        
        loss = self.forward_with_loss_calculation(tokens_tensor, sent1_idx, sent2_idx, orig_to_tok_map, l)

        return {'val_loss': loss}
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        # return torch.optim.SGD(self.parameters(), lr=0.005, momentum=0.9)
        return torch.optim.Adam(self.parameters(), weight_decay=1e-4)
    
    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen
    
    
def get_prediction(sent1, sent2, model):

    l = len(sent1.split(" ")) + 1 
    sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
    sents_concat = sent1 + " ***** " + sent2
    sent1_arg1 = [i for i,w in enumerate(sent1.split(" ")) if "ARG1:" in w][0]
    sent1_arg2 = [i for i,w in enumerate(sent1.split(" ")) if "ARG2:" in w][0]
    
    bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
    outputs = model.model(tokens_tensor)
    states = outputs[0][0] #[seq_len, 768]
    states = states/torch.norm(states, dim = 1, keepdim = True)
    sent1_arg1_vec, sent1_arg2_vec = states[orig_to_tok_map[sent1_arg1]], states[orig_to_tok_map[sent1_arg2]]
    
    sims_arg1 = sent1_arg1_vec @ states.T
    sims_arg2 = sent1_arg2_vec @ states.T
    
    idx_arg1 = torch.argsort(sims_arg1)
    idx_arg2 = torch.argsort(sims_arg2)
        
    most_sim_arg1, most_sim_arg2 = sims_arg1[idx_arg1[-2]], sims_arg2[idx_arg2[-2]]
    ind_arg1, ind_arg2 = idx_arg1[-2].detach().cpu().numpy().item(), idx_arg2[-2].detach().cpu().numpy().item()
    return sents_concat, tok_to_orig_map[ind_arg1], tok_to_orig_map[ind_arg2], bert_tokens[ind_arg1], bert_tokens[ind_arg2]

In [8]:
class Dataset(torch.utils.data.Dataset):
    """Simple torch dataset class"""

    def __init__(self, data: List[Dict], device = "cpu"):

        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        with torch.no_grad():
            
            d = self.data[index]
            sent1, sent2 = d["first"], d["second"]
            sent1_arg1, sent1_arg2 = d["first_arg1"][0], d["first_arg2"][0]
            sent2_arg1, sent2_arg2 = d["second_arg1"][0], d["second_arg2"][0]
            
            l = len(sent1.split(" ")) + 1 
            sent2_arg1, sent2_arg2 = sent2_arg1 + l, sent2_arg2 + l

            sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
            sents_concat = sent1 + " ***** " + sent2 #sents_concat.split(" ")[l] is the first token in the 2nd sent
            
            return sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l

In [9]:
l = int(0.8 * len(data))
train_dataset, dev_dataset = Dataset(data[:l], "cpu"), Dataset(data[l:], "cpu")

In [29]:
sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l = train_dataset[5]

100

In [16]:
sents_concat

'Steroids are ARG1:the first-line front treatment of ICI myocarditis , despite poor evidence to support this , followed by other ARG2:immune modulators , plasma exchange , or even CTLA-4 agonist ( abatacept ) infusions . ***** Moreover , the basal ECAR observed among Tyto sorted cells was similar to that 212 observed in immune populations in our previous publication [ 5 ] .'

In [30]:
del model

In [10]:
model = BertModel(train_dataset, dev_dataset, 1, "cuda", "train")

In [19]:
bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
indexed_tokens = model.tokenizer.convert_tokens_to_ids(bert_tokens)
tokens_tensor = torch.tensor([indexed_tokens])

In [20]:
model.forward_with_loss_calculation(tokens_tensor, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), orig_to_tok_map, l)

tensor([0.8575], grad_fn=<AddBackward0>)

In [11]:
#model.to("cpu")
#model.device = "cpu"
trainer = Trainer(max_nb_epochs=1,min_nb_epochs=1, gpus = 1)
trainer.fit(model)

2020-10-19 21:37:43.692 INFO    lightning: GPU available: True, used: True
2020-10-19 21:37:43.693 INFO    lightning: CUDA_VISIBLE_DEVICES: [0]
/home/shauli/miniconda3/envs/py3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
2020-10-19 21:37:56.521 INFO    lightning: 
    | Name                                              | Type              | Params
------------------------------------------------------------------------------------
0   | model                                             | BertModel         | 109 M 
1   | model.embeddings                                  | BertEmbeddings    | 24 M  
2   | model.embeddings.word_embeddings                  | Embedding         | 23 M  
3   | model.embeddings.position_embeddings              | Embedding        

/home/shauli/miniconda3/envs/py3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Loss is 0.9093616604804993


/home/shauli/miniconda3/envs/py3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


2020-10-19 21:45:02.731 INFO    lightning: Detected KeyboardInterrupt, attempting graceful shutdown...


1

In [82]:
k = 1
sents_concat, ind_arg1, ind_arg2, bert_tok_arg1, bert_tok_arg2 = get_prediction(data[k]["first"], data[k]["second"], model)

3


In [83]:
print(sents_concat.split(" ")[ind_arg1])
print(sents_concat.split(" ")[ind_arg2])

print(bert_tok_arg1)
print(bert_tok_arg2)

genes
HEK
genes
hek


In [84]:
sents_concat

'For example , Fornek et al. showed that the H5N1 influenza ARG1:A/Hong Kong/486/97 virus containing amino acid substitution E627 K in PB2 upregulated TCR ARG2:complex genes ( e.g. , CD3D and CD3 G ) in the lungs of infected mice at day 2 post-infection [ 11 ] . ***** In a third approach , the adeno-associated virus rep protein mediates stable integration of antibody genes into HEK cells .'